In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as tud

from collections import Counter
import numpy as np
import random
import math

import pandas as pd
import scipy
from sklearn.metrics.pairwise import cosine_similarity

USE_CUDA = torch.cuda.is_available()

random.seed(1)
np.random.seed(1)
torch.manual_seed(1)
if USE_CUDA:
    torch.cuda.manual_seed(1)

C = 3
K = 100
NUM_EPOCHS = 2
MAX_VACB_SIZE = 30000
BATCH_SIZE = 128
LEARNING_RATE = 0.2
EMBEDDING_SIZE = 100

def word_tokenize(text):
    return text.split()

In [2]:
with open('text8/text8.train.txt') as fin:
    text = fin.read()
    
text = text.split()
vocab = dict(Counter(text).most_common(MAX_VACB_SIZE - 1))
vocab['<unk>'] = len(text) - np.sum(list(vocab.values()))

idx_to_word = [word for word in vocab.keys()]
word_to_idx = {word:i for i, word in enumerate(idx_to_word)}

In [3]:
list(word_to_idx.items())[:10]

[('the', 0),
 ('of', 1),
 ('and', 2),
 ('one', 3),
 ('in', 4),
 ('a', 5),
 ('to', 6),
 ('zero', 7),
 ('nine', 8),
 ('two', 9)]

In [4]:
word_counts = np.array([count for count in vocab.values()], dtype=np.float32)
word_freqs = word_counts / np.sum(word_counts)
word_freqs = word_freqs ** (3./4.)
word_freqs = word_freqs / np.sum(word_freqs)
VOCAB_SIZE = len(idx_to_word)
VOCAB_SIZE

30000

In [5]:
class WordEmbeddingDataset(tud.Dataset):
    def __init__(self, text, word_to_idx, idx_to_word, word_freqs, word_counts):
        super(WordEmbeddingDataset, self).__init__()
        self.text_encoded = [word_to_idx.get(word, word_to_idx['<unk>']) for word in text]
        self.text_encoded = torch.LongTensor(self.text_encoded)
        self.word_to_idx = word_to_idx
        self.idx_to_word = idx_to_word
        self.word_freqs = torch.Tensor(word_freqs)
        self.word_counts = torch.Tensor(word_counts)
    
    def __len__(self):
        return len(self.text_encoded)

    def __getitem__(self, idx):
        center_word = self.text_encoded[idx]
        pos_indices = list(range(idx-C, idx)) + list(range(idx+1, idx+C+1))
        pos_indices = [i % len(self.text_encoded) for i in pos_indices]
        pos_words = self.text_encoded[pos_indices]
        neg_words = torch.multinomial(self.word_freqs, K * pos_words.shape[0], True)
        return center_word, pos_words, neg_words

In [6]:
dataset = WordEmbeddingDataset(text, word_to_idx, idx_to_word, word_freqs, word_counts)
dataloader = tud.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [7]:
next(iter(dataloader))

[tensor([  819,    45,   621,    15,  1797,    29,   328,   157,    25,   598,
             9,    13,    25,    12,     5,  4344,     3,    13,     0,     5,
          1532,   648,     9,   937,    16, 22599,    85,  7406,  2801,   419,
          1238,     1,   966,  1655,   644,     6,    16, 18573, 11226,    37,
           261,  1514,  3537,     1, 29999,   644,     4,   210,   110,     5,
          3316,  1454,    29,     7,     0,   825,     2,  3992,  2991,  9029,
          1881,     0, 20161,    13,     5,     4, 12028,  7117,   394,     3,
         27580,  3642,    36,  2050,    92,     8, 23976,  2184,   335,   339,
          1314,    15,    34,   284,  4247,  2389,    25,  8552,     0,  1467,
           131,  5437,     1, 10596,     2,     4,  1963,    37,     5,   401,
          2111,     2,     6,     0,    14,    10,     1,     9,  5363, 12439,
          8464,     1,   432,   298,  4171, 11035,     0,  3513,     4,   969,
            28,   836, 29999,    88,     5,     6,  

In [8]:
class EmbeddingModel(nn.Module):
    def __init__(self, vocab_size, embed_size):
        super(EmbeddingModel, self).__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        
        initrange = 0.5 / self.embed_size
        self.in_embed = nn.Embedding(self.vocab_size, self.embed_size)
        self.in_embed.weight.data.uniform_(-initrange, initrange)
        self.out_embed = nn.Embedding(self.vocab_size, self.embed_size)
        self.out_embed.weight.data.uniform_(-initrange, initrange)

    def forward(self, input_labels, pos_labels, neg_labels):
        input_embedding = self.in_embed(input_labels)
        pos_embedding = self.out_embed(pos_labels)        
        neg_embedding = self.out_embed(neg_labels)        
        
        input_embedding = input_embedding.unsqueeze(2)
        pos_dot = torch.bmm(pos_embedding, input_embedding).squeeze(2)
        neg_dot = torch.bmm(neg_embedding, -input_embedding).squeeze(2)        
        
        log_pos = F.logsigmoid(pos_dot).sum(1)
        log_neg = F.logsigmoid(neg_dot).sum(1)
        
        loss = log_pos + log_neg
        return -loss
    
    def input_embedding(self):
        return self.in_embed.weight.data.cpu().numpy()

In [9]:
model = EmbeddingModel(VOCAB_SIZE, EMBEDDING_SIZE)
if USE_CUDA:
    model = model.cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)
for e in range(NUM_EPOCHS):
    for i, (input_labels, pos_labels, neg_labels) in enumerate(dataloader):
        input_labels = input_labels.long()
        pos_labels = pos_labels.long()        
        neg_labels = neg_labels.long()      
        if USE_CUDA:
            input_labels = input_labels.cuda()
            pos_labels = pos_labels.cuda()        
            neg_labels = neg_labels.cuda() 
        optimizer.zero_grad()
        loss = model(input_labels, pos_labels, neg_labels).mean()
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print('epoch', e, 'iteration', i, loss.item())

epoch 0 iteration 0 420.04705810546875
epoch 0 iteration 10 420.02923583984375


KeyboardInterrupt: 